<a href="https://colab.research.google.com/github/NIXBLACK11/ModelForAnalysis/blob/main/VideoRating2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
! rm -rf screenshot audio

In [3]:
from moviepy.video.io.VideoFileClip import VideoFileClip
import os
from PIL import Image

audioIndex = 0
imageIndex = 0
totalTime = 0

def extract_screenshots_and_audio(input_video_path, output_screenshots_folder, output_audio_folder):
    global audioIndex, imageIndex, totalTime

    # Create output folders if they don't exist
    os.makedirs(output_screenshots_folder, exist_ok=True)
    os.makedirs(output_audio_folder, exist_ok=True)

    # Load video
    video_clip = VideoFileClip(input_video_path)

    # Extract screenshots and calculate mean time duration
    screenshots = []
    mean_time_duration = 0
    for timestamp in range(0, int(video_clip.duration), 5):
        screenshot = video_clip.get_frame(timestamp)

        # Convert NumPy array to PIL Image
        pil_image = Image.fromarray(screenshot)

        screenshots.append(pil_image)
        mean_time_duration += timestamp

        screenshot_path = os.path.join(output_screenshots_folder, f"screenshot_{imageIndex}.png")
        imageIndex += 1
        pil_image.save(screenshot_path)

    totalTime += video_clip.duration

    # Extract audio
    audio = video_clip.audio
    audio_path = os.path.join(output_audio_folder, f"audio_{audioIndex}.wav")
    audio.write_audiofile(audio_path)

    audioIndex += 1  # Increment audio index for unique filenames

    # Close video clip
    video_clip.close()

if __name__ == "__main__":
  genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

  for genre in genres:
      input_video_path = "/content/drive/MyDrive/VideoDataset/"+genre
      output_screenshots_folder = "/content/screenshot/"+genre
      output_audio_folder = "/content/audio/"+genre

      totalFiles = 0

      for filename in os.listdir(input_video_path):
        # Construct the full file path
        input_video_file = os.path.join(input_video_path, filename)

        # Check if it's a file (not a subdirectory)
        if os.path.isfile(input_video_file):
            # Call the function with the file path
            extract_screenshots_and_audio(input_video_file, output_screenshots_folder, output_audio_folder)
            totalFiles += 1

      averageTime = totalTime / totalFiles
      print(averageTime)


KeyboardInterrupt: 

In [ ]:
genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Set the paths
base_dir = '/content/screenshot'

# Create a dictionary to map genres to their respective folders
genre_folders = {
    'MrBeastType': 'A',
    'VlogType': 'B',
    'TechReviewType': 'C',  # Change 'C' to the actual folder name for TechReviewType
    'GamingType': 'D',  # Change 'D' to the actual folder name for GamingType
    'MinimalistType': 'E'  # Change 'E' to the actual folder name for MinimalistType
}

# Get the list of image files for each genre
genre_images = {}
for genre, folder in genre_folders.items():
    genre_dir = os.path.join(base_dir, folder)
    genre_images[genre] = [os.path.join(genre_dir, img) for img in os.listdir(genre_dir)]

# Choose an equal number of images from all classes
num_images_per_genre = min(len(images) for images in genre_images.values())
for genre in genre_images:
    genre_images[genre] = genre_images[genre][:num_images_per_genre]

# Split the data into train and test sets for each genre
train_images, test_images = {}, {}
for genre, images in genre_images.items():
    train_images[genre], test_images[genre] = train_test_split(images, test_size=0.2, random_state=42)

# Create train and test directories for each genre
train_dir = '/content/dataset/train/'
test_dir = '/content/dataset/test/'

for genre in genres:
    os.makedirs(os.path.join(train_dir, genre), exist_ok=True)
    os.makedirs(os.path.join(test_dir, genre), exist_ok=True)

# Copy an equal number of images to train and test directories for each genre
for genre in genres:
    for img in train_images[genre]:
        os.system(f'cp "{img}" "{os.path.join(train_dir, genre)}"')

    for img in test_images[genre]:
        os.system(f'cp "{img}" "{os.path.join(test_dir, genre)}"')

# Set up image data generators for train and test sets
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multiple classes
    classes=genres
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multiple classes
    classes=genres
)


In [ ]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pickle

# Function to create, compile, train, and save model with 'adamax' optimizer
def train_and_save_model(train_generator, test_generator):
    # Load the VGG16 model and remove the final layer
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(len(genres), activation='softmax')(x)  # Use 'softmax' for multiple classes

    # Create a new model by combining the VGG16 base model and the new layers
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(genres), activation='softmax'))  # Use 'softmax' for multiple classes

    # Freeze the VGG16 base layers and train the new layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])  # Use 'categorical_crossentropy' for multiple classes

    print("\nTraining model with optimizer: adamax")

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=20,
        validation_data=test_generator,
        validation_steps=test_generator.samples // batch_size
    )

    # Save the model
    model.save('/content/drive/MyDrive/VideoDataset/video_analysis_vgg16_adamax.h5')

    # Save the training history to a file
    history_path = '/content/drive/MyDrive/VideoDataset/training_history_adamax.pkl'
    with open(history_path, 'wb') as file:
        pickle.dump(history.history, file)

    return history

# Set up image data generators for train and test sets
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multiple classes
    classes=genres
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multiple classes
    classes=genres
)

# Train and save model with 'adamax' optimizer
history = train_and_save_model(train_generator, test_generator)

# Access training metrics for plotting
print(f'Model with optimizer adamax - Final Training Accuracy: {history.history["accuracy"][-1]}, Final Validation Accuracy: {history.history["val_accuracy"][-1]}')


In [ ]:
from tensorflow.keras.models import load_model

# Evaluate the model with 'adamax' optimizer
optimizer_name = 'adamax'
model_path = f'/content/drive/MyDrive/VideoDataset/video_analysis_vgg16_{optimizer_name}.h5'
model = load_model(model_path)

# Evaluate the model on the "test" set
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)

print(f'Model with optimizer {optimizer_name} - Test accuracy: {test_acc}, Test loss: {test_loss}')


In [ ]:
import matplotlib.pyplot as plt

# Create a figure with subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
fig.suptitle('Training and Validation Metrics')

# Evaluate the model with 'adamax' optimizer
optimizer_name = 'adamax'
model_path = f'/content/drive/MyDrive/VideoDataset/video_analysis_vgg16_{optimizer_name}.h5'
model = load_model(model_path)

# Load training history
history_path = f'/content/drive/MyDrive/VideoDataset/training_history_{optimizer_name}.pkl'  # Replace with the actual path
with open(history_path, 'rb') as file:
    history = pickle.load(file)

# Plot accuracy
ax1.plot(history['accuracy'], label=f'{optimizer_name} Training Accuracy')
ax1.plot(history['val_accuracy'], label=f'{optimizer_name} Validation Accuracy')

# Plot loss
ax2.plot(history['loss'], label=f'{optimizer_name} Training Loss')
ax2.plot(history['val_loss'], label=f'{optimizer_name} Validation Loss')

# Set labels and title
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True)

ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

plt.show()


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

# Function to preprocess an image for prediction
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the pixel values to be between 0 and 1
    return img_array, img

# Choose a random image from the test folder
test_images_folder = '/content/dataset/test'
genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']
random_genre = random.choice(genres)
random_image_name = random.choice(os.listdir(os.path.join(test_images_folder, random_genre)))
random_image_path = os.path.join(test_images_folder, random_genre, random_image_name)

# Preprocess the image
img, original_img = preprocess_image(random_image_path)

# Load the 'adamax' model
optimizer_name = 'adamax'
model_path = f'/content/drive/MyDrive/VideoDataset/video_analysis_vgg16_{optimizer_name}.h5'
model = load_model(model_path)

# Make a prediction
prediction = model.predict(img)

# Display the result along with the actual image and selected genre
plt.imshow(original_img)
plt.axis('off')

print(f'Model with optimizer {optimizer_name}:')
print(f'Selected from genre: {random_genre}')
predicted_genre = genres[np.argmax(prediction)]
plt.title(f'Predicted: {predicted_genre}')
plt.show()
print('---')


In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from PIL import Image

audioIndex = 0
imageIndex = 0
totalTime = 0

# Load the 'adamax' model
optimizer_name = 'adamax'
model_path = '/content/drive/MyDrive/VideoDataset/video_analysis_vgg16_adamax.h5'  # Replace with the actual path
model = load_model(model_path)

def preprocess_image(img):
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the pixel values to be between 0 and 1
    return img_array

def predict_genre(img):
    img_array = preprocess_image(img)
    prediction = model.predict(img_array)
    predicted_genre = genres[np.argmax(prediction)]
    return predicted_genre

def extract_screenshots_and_audio(input_video_path, output_screenshots_folder, output_audio_folder):
    global audioIndex, imageIndex, totalTime

    # Create output folders if they don't exist
    os.makedirs(output_screenshots_folder, exist_ok=True)
    os.makedirs(output_audio_folder, exist_ok=True)

    # Load video
    video_clip = VideoFileClip(input_video_path)

    # Extract screenshots and calculate mean time duration
    screenshots = []
    mean_time_duration = 0
    for timestamp in range(0, int(video_clip.duration), 5):
        screenshot = video_clip.get_frame(timestamp)

        # Convert NumPy array to PIL Image
        pil_image = Image.fromarray(screenshot)

        # Predict the genre using the 'adamax' model
        predicted_genre = predict_genre(pil_image)

        # Save screenshot with predicted genre as the filename
        screenshot_path = os.path.join(output_screenshots_folder, f"{predicted_genre}_{imageIndex}.png")
        imageIndex += 1
        pil_image.save(screenshot_path)

        screenshots.append(pil_image)
        mean_time_duration += timestamp

    totalTime += video_clip.duration

    # Extract audio
    audio = video_clip.audio
    audio_path = os.path.join(output_audio_folder, f"audio_{audioIndex}.wav")
    audio.write_audiofile(audio_path)

    audioIndex += 1  # Increment audio index for unique filenames

    # Close video clip
    video_clip.close()

if __name__ == "__main__":
    input_video_path = "/content/drive/MyDrive/VideoDataset/TestVideos/videoplayback (1).mp4"
    output_screenshots_folder = "/content/Test/screenshot"
    output_audio_folder = "/content/Test/audio"

    genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

    # Call the function with the video file path
    extract_screenshots_and_audio(input_video_path, output_screenshots_folder, output_audio_folder)

    # Calculate average time (if needed)
    averageTime = totalTime
    print(averageTime)


In [ ]:
import os
import numpy as np
from keras.preprocessing import image
from keras.models import load_model

# Function to preprocess an image for prediction
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the pixel values to be between 0 and 1
    return img_array, img

# Function to predict the genre using the 'adamax' model
def predict_genre(img):
    # Load the 'adamax' model
    optimizer_name = 'adamax'
    model_path = f'/content/drive/MyDrive/VideoDataset/video_analysis_vgg16_{optimizer_name}.h5'
    model = load_model(model_path)

    # Make a prediction
    prediction = model.predict(img)

    if prediction[0][0] > 0.5:
        return "Mr.BeastType"
    else:
        return "MinimalistType"

# Initialize counters
totalBeastVideos = 0
totalMinVideos = 0

# Loop through all screenshots
for image_name in os.listdir("/content/Test/screenshot"):
    # Preprocess the image
    image_path = os.path.join("/content/Test/screenshot", image_name)
    img, original_img = preprocess_image(image_path)

    # Predict the genre using the 'adamax' model
    predicted_genre = predict_genre(img)

    # Update counters
    if predicted_genre == 'Mr.BeastType':
        totalBeastVideos += 1
    else:
        totalMinVideos += 1

# Print the overall genre based on majority vote
if totalBeastVideos > totalMinVideos:
    print("Overall Genre: Mr.BeastType")
else:
    print("Overall Genre: MinimalistType")


In [ ]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

# Function to extract MFCC features from audio files
def extract_features(file_path):
    y, sr = librosa.load(file_path, mono=True, duration=5)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs

# List of genres
genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

# List to store features and labels
features = []
labels = []

# Load data for each genre
for i, genre in enumerate(genres):
    genre_path = f'/content/audio/{genre}'
    for file_name in os.listdir(genre_path):
        file_path = os.path.join(genre_path, file_name)
        mfccs = extract_features(file_path)
        features.append(mfccs)
        labels.append(i)  # Assign a unique label for each genre

# Convert lists to NumPy arrays
X = np.array(features)
y = np.array(labels)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Transpose the MFCCs before reshaping
X_train = X_train.transpose(0, 2, 1)
X_val = X_val.transpose(0, 2, 1)
X_test = X_test.transpose(0, 2, 1)

# Reshape the input data to match Conv1D input shape
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], X_val.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Define and compile the model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(genres), activation='softmax'))  # Output layer with softmax activation for multiple genres
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

# Save the trained model
model.save('/content/drive/MyDrive/VideoDataset/audio_analysis_multi_genre.h5')


In [ ]:
import os
import librosa
import numpy as np
from keras.models import load_model

# Function to extract MFCC features from audio files
def extract_features(file_path):
    y, sr = librosa.load(file_path, mono=True, duration=5)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs

# Load the trained model
model_path = '/content/drive/MyDrive/VideoDataset/audio_analysis_multi_genre.h5'  # Use the model trained for multiple genres
model = load_model(model_path)

# New audio file path
new_audio_path = '/content/Test/audio/audio_0.wav'

# Extract features from the new audio file
new_audio_features = extract_features(new_audio_path)

# Transpose and reshape the features
new_audio_features = new_audio_features.transpose(1, 0)  # Transpose
new_audio_features = new_audio_features.reshape(1, new_audio_features.shape[0], new_audio_features.shape[1])

# Predict the class
prediction = model.predict(new_audio_features)

# Map the predicted class to a label
genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']
predicted_class = genres[np.argmax(prediction)]

# Print the prediction
print(f"Predicted Genre: {predicted_class}")
